 # Competición titanic con keras


 ## importar librerias y datos

In [1]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [129]:
item_categories = pd.read_csv('data/item_categories.csv')
items = pd.read_csv('data/items.csv')
sales_train = pd.read_csv('data/sales_train.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')
shops = pd.read_csv('data/shops.csv')
test = pd.read_csv('data/test.csv')

datasets = {
    'item_categories': item_categories.columns.values,
    'items': items.columns.values,
    'sales_train': sales_train.columns.values,
    'sample_submission': sample_submission.columns.values,
    'shops': shops.columns.values,
    'test': test.columns.values
}

# Si cada dataset tiene menos de 6 columnas, completar con NaN.
for dataset in datasets:
    while len(datasets[dataset]) < 6:
        datasets[dataset] = np.append(datasets[dataset], '-')

pd.DataFrame(datasets)

,item_categories,items,sales_train,sample_submission,shops,test
0,item_category_name,item_name,date,ID,shop_name,ID
1,item_category_id,item_id,date_block_num,item_cnt_month,shop_id,shop_id
2,-,item_category_id,shop_id,-,-,item_id
3,-,-,item_id,-,-,-
4,-,-,item_price,-,-,-
5,-,-,item_cnt_day,-,-,-


In [142]:
# ds['date'] = pd.to_datetime(ds['date'], format='%d.%m.%Y')
# ds['day'] = ds.date.dt.day
# ds['month'] = ds.date.dt.month
# ds['year'] = ds.date.dt.year

In [164]:
ds = sales_train

ds.rename(columns={'item_cnt_day':'sales', 'date_block_num':'block'}, inplace=True)
a = ds.groupby(['block','shop_id','item_id']).sum()['sales'].reset_index()
b = ds.groupby(['block','shop_id','item_id']).mean()['item_price'].reset_index()
a.insert(a.shape[-1]-1, 'mean_monthly_price', b.item_price)
ds = a.sort_values(by=['item_id', 'shop_id', 'block'])

ds.loc[ds['item_id']==33]

,block,shop_id,item_id,mean_monthly_price,sales
1,0,0,33,347.0,3.0
63227,1,0,33,347.0,3.0
65663,1,1,33,347.0,1.0
3921,0,2,33,499.0,1.0
569988,10,2,33,199.0,2.0
...,...,...,...,...,...
620580,10,59,33,199.0,1.0
740199,12,59,33,199.0,2.0
927875,16,59,33,199.0,1.0
1254395,23,59,33,199.0,1.0


In [ ]:
x_train = sales_train.groupby(['item_id']).sum()['item_cnt_day'].reset_index()

# Dataset a meses

In [14]:
x_train.tail().style.background_gradient()

,date_block_num,shop_id,item_id,item_cnt_day
1609119,33,59,22087,6.000000
1609120,33,59,22088,2.000000
1609121,33,59,22091,1.000000
1609122,33,59,22100,1.000000
1609123,33,59,22102,1.000000


In [10]:
x_train.head()

,item_id,item_cnt_day
0,0,1.0
1,1,6.0
2,2,2.0
3,3,2.0
4,4,1.0


In [15]:
x_train.shape

(1609124, 4)

 ## Ingeniera de datos

In [126]:
y_train = x_train.item_cnt_day
x_train.drop(['item_cnt_day'], axis=1, inplace=True)


# data[:891].Survived

# # Borrar columnas innesesarias.
# features_to_remove = ['PassengerId', 'Ticket', 'Cabin',  'Survived', 'Fare', ]
# data = data.drop(features_to_remove , axis=1)
# #  Reemplazar datos categóricos por de sexo 0s y  por 1s.
# data['Sex'] = LabelEncoder().fit_transform(data['Sex'])

 ### Generación de nuevos atributos

 A partir de ``SibSp`` y ``Parch`` generamos y reemplazamos lo anterior con un nuevo atributo para verificar si el pasajero arrivó solo o con algún pariente: ``IsAlone``.

In [5]:
# data['IsAlone'] = 0
# data.loc[ (data['SibSp'] + data['Parch']) == 0, 'IsAlone'] = 1 
# data.drop(['SibSp', 'Parch'], axis=1, inplace=True)

 A partir de Los prefijos de los nombres del los pasajeros, se genera un nuevo atributo que representa el prefijo de los nombres: ``Title``.


In [6]:
# Extraer prefijo del nombre de las personas.
# data['Title'] = data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

# Reemplazar los titulos raros con uno mas abarcativo: 'Rare'.
# data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
# data['Title'] = data['Title'].replace(['Mlle','Ms'], 'Miss')
# data['Title'] = data['Title'].replace('Mme', 'Mrs')
# data['Title'] = LabelEncoder().fit_transform(data['Title'])
# data.drop(['Name'], axis=1, inplace=True)


 Rellenamos ``Age`` faltantes según la relación con ``Sex`` y ``Pclass``.

In [7]:
# guess_ages = np.zeros((2,3))

# for i in range(0, 2): # itera con => 0,1 para sexo
#     for j in range(0, 3):  # itera con => 1,2,3 para pclass
#         guess_df = data[(data['Sex'] == i) & (data['Pclass'] == j+1)]['Age'].dropna()

#         # Convierte el numero decimal al .5 mas cercano
#         guess_ages[i,j] = round( guess_df.mean() *2 ) / 2

#         data.loc[ (data.Age.isnull()) & (data.Sex == i) & (data.Pclass == j+1),'Age'] = guess_ages[i,j]

# data["AgeCat"]= pd.cut(data["Age"], bins=[0, 15, 30, 45, 60, max(data["Age"]+1)], labels=[1, 2, 3, 4, 5])
# data.drop(['Age'], axis=1, inplace=True)


 rellamos ``Embarked`` con el valor mas comun

In [8]:
# data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

 Convertimos columna de datos categóricos a variables dummys.

In [9]:
# data = pd.get_dummies(data, columns=['Pclass', 'Embarked', 'Title'], prefix=['Pclass','Embarked', 'Title'])#, drop_first=True)

 Split y visualización de ``data``.

In [10]:
# X_train = data[:891]
# test = data[891:]

 ## Definición de modelo

In [3]:
def build_model(lr):

  model = keras.Sequential([
    keras.layers.LSTM(units=32, input_shape=(1,3)),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation='relu') 
  ])

  model.compile(loss='mse',
                optimizer=Adam(learning_rate=lr),
                metrics=['accuracy'])

  return model

 ## Entrenamiento del modelo


### Instaciacion del modelo.

In [6]:
keras.utils.set_random_seed(0)
model = build_model(0.1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 32)                4608      
                                                                 
 batch_normalization_2 (Batc  (None, 32)               128       
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,769
Trainable params: 4,705
Non-trainable params: 64
_________________________________________________________________
None


### entrenamiento normal

In [ ]:
history = model.fit(
    x_train,
    y_train,
    verbose=2, epochs=25, batch_size=x_train.shape[0])

 ## Evaluar con data 100% accuracy
 para evitar tener que estar subiendo constantemente la submission a kaggle, lo comparamos con el resultado final de la competición que se encuentra en un repo en github.

In [14]:
y_test = pd.read_csv('data/submission_100accuracy.csv')['Survived']

model.evaluate(test, y_test)

14/14 [==============================] - 0s 2ms/step - loss: 0.5080 - accuracy: 0.7871


[0.5079693794250488, 0.7870813608169556]

 ## Predicciones

In [188]:
preds = model.predict(test)
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0

14/14 [==============================] - 0s 2ms/step


 ## Exportar predicciones

In [189]:
prediction = pd.DataFrame({'PassengerId': test_data.PassengerId.values, 'Survived': preds.ravel().astype('int64')})
prediction.to_csv('data/results.csv', index=False)

In [190]:
prediction.head(5)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
